In [1]:
library(hise)
library(flowCore)
library(dplyr)
library(glue)


Attaching package: ‘dplyr’


The following object is masked from ‘package:flowCore’:

    filter


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [72]:
# add metadata from LIMS
LIMS_meta <- data.table::fread("/home/jupyter/FH3/flow/Content_20220315_122909_subjectGUIDs_PTIDs.txt")
LIMS_meta <- select(LIMS_meta, c("cntn_cf_patient", "cntn_cf_externalPatientID")) %>% distinct()
lims_mat <- dplyr::right_join(LIMS_meta, mat, by = c("cntn_cf_patient" = "subject.subjectGuid"))


In [77]:
# filter for samples in orig manuscript
manuscript_meta <- data.table::fread("/home/jupyter/FH3/flow/Fig-S1c-Flow-PCA_fromSVV.txt")
manu_mat <- lims_mat %>% dplyr::filter(cntn_cf_externalPatientID %in% manuscript_meta$patientID)
c <- manuscript_meta[,c(1:6,9:19)]
c1 <- filter(c, Sex %in% c("male", "female"))

PCA_samples <- filter(c1, Status == "Covid" & Age > 44)

In [78]:
# add clinical meta
add_meta <- data.table::fread("/home/jupyter/FH3/flow/COVID-Longitudinal-metadata_fromSVV.txt")
add_meta$PTID <- as.character(add_meta$PTID)
add_meta$visit <- as.character(add_meta$visit)
manu_mat$visit <- stringr::str_sub(manu_mat$sample.visitName, start= -1) 
fullmeta_mat <- dplyr::left_join(manu_mat, add_meta, 
                                  c("cntn_cf_externalPatientID" = "PTID",
                                   "visit" = "visit"))

long_samples <- filter(add_meta, age_bin == "olderadults" & Status == "Covid")



V1,PTID_visit,PTID,visit,FH.Subject.ID,Status,Healthy_consider,scRNA_available,Subject,Visit,⋯,StotpctMBCslope,StotpctMBCday30,StotpctMBCday180,SIgGpctMBCintercept,SIgGpctMBCslope,SIgGpctMBCday30,SIgGpctMBCday180,nAb.GMT,Stage,age_bin
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
32251_1,32251_1,32251,1,32251,Covid,0,1,FH3043,FH3 COVID-19 Visit 1,⋯,NA,NA,NA,NA,NA,NA,NA,10,Early,olderadults


In [ ]:
# check that all samples are present
check <- select(fullmeta_mat, c("cntn_cf_patient", "subject.biologicalSex", "Status", "age_bin", "Age")) %>% 
            filter(startsWith(fullmeta_mat$cntn_cf_patient, "FH")) %>% distinct()

table(check$subject.biologicalSex, check$Status)
table(check$subject.biologicalSex, check$age_bin)
check %>% group_by(Status, age_bin) %>% mutate(median_age = median(Age, na.rm = TRUE)) %>%
        select(Status, age_bin, median_age) %>% distinct()

checkg <- fullmeta_mat %>% filter(startsWith(fullmeta_mat$cntn_cf_patient, "FH")) %>% 
            distinct() %>% select(Status, age_bin, cntn_cf_patient, visit) %>% distinct()

table(checkg$age_bin, checkg$Status)

# c <- filter(checkg, age_bin == "olderadults" & Status == "Covid")
# table(c$visit)

        
         Covid Healthy
  Female    10      10
  Male       8      13

        
         olderadults youngeradults
  Female          12             8
  Male            15             6

Status,age_bin,median_age
<chr>,<chr>,<dbl>
Healthy,olderadults,54
Healthy,youngeradults,34
Covid,olderadults,57
Covid,youngeradults,29


               
                Covid Healthy
  olderadults      43      16
  youngeradults    26       7

In [ ]:
# download fcs files
panels <- c("PM1", "PB1", "PT1")
for (i in 1:length(panels)){
    mat <- matrix(nrow = 0, ncol = 23)
    panel = panels[i]

    download_path <- glue::glue("/home/jupyter/FH3/flow/fcs/{panel}")
    setwd(dir=download_path)
    
    filter_list <- list(
        cohort.cohortGuid = c("FH3"),
        file.panel = panel
    )
    
    filedescdf <- hise::getFileDescriptors(fileType = "FlowCytometry",filter = filter_list) %>% 
            hise::fileDescToDataframe()
    mat <- rbind(mat, filedescdf)
    
    filter_list <-  list(subject.subjectGuid = "HMN169517",
                    file.panel = panels[i])
    filedescdf_bc <- hise::getFileDescriptors(fileType = "FlowCytometry",filter = filter_list) %>% 
            hise::fileDescToDataframe()
    mat <- rbind(filedescdf, filedescdf_bc)
    
    filter_mat <- mat %>% dplyr::filter(sample.id %in% fullmeta_mat$sample.id)
    flowfiles_cache <-hise::cacheFiles(list(filter_mat$file.id))
}